In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [13]:
train = pd.read_csv('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/train.csv')
test = pd.read_csv('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/test.csv')
submit = pd.read_csv('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/submit_file.csv')

In [19]:
len(train['occyp_type'].unique())

19

#**Data Preprocessing**


1.	문자로 된 것들은 다 라벨인코딩

2.	Child_num은 이대로 사용

3.	Income_total /10000으로 스케일링 진행

4.	['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree'] 4 3 2 1 5로 라벨인코딩

5.	Minus 값들은 베이스라인 코드 따기

6.	Occupy_type NaN값은 0 나머진 label encoder

7.	FLAG MOBIL 제거

8.	민영 – LightGBM, pycarot / 채원 – Catboost, XGBoost/ 영민 – Randomforest, logistic regression / SVM ensemble


In [20]:
# index 와 FLAG_MOBIL 제거
train = train.drop('index',axis=1)
train = train.drop('FLAG_MOBIL',axis=1)

In [21]:
#문자로 된것들 라벨인코딩

train['gender'] = train['gender'].replace(['F','M'],[0,1])
test['gender'] = test['gender'].replace(['F','M'],[0,1])
print('gender :')
print(train['gender'].value_counts())
print('--------------')

print('Having a car or not : ')
train['car'] = train['car'].replace(['N','Y'],[0,1])
test['car'] = test['car'].replace(['N','Y'],[0,1])
print(train['car'].value_counts())
print('--------------')

print('Having house reality or not: ')
train['reality'] = train['reality'].replace(['N','Y'],[0,1])
test['reality'] = test['reality'].replace(['N','Y'],[0,1])
print(train['reality'].value_counts())
print('--------------')
       
train['edu_type'] = train['edu_type'].replace([ 'Academic degree','Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary'],[4,3,2,1,0])
test['edu_type'] = test['edu_type'].replace([ 'Academic degree','Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary'],[4,3,2,1,0])

gender :
0    17697
1     8760
Name: gender, dtype: int64
--------------
Having a car or not : 
0    16410
1    10047
Name: car, dtype: int64
--------------
Having house reality or not: 
1    17830
0     8627
Name: reality, dtype: int64
--------------


In [22]:
# income total 전처리

train['income_total'] = train['income_total'].astype(object)
train['income_total'] = train['income_total']/10000 
test['income_total'] = test['income_total']/10000

In [23]:
count, bin_dividers =np.histogram(train['income_total'], bins=7)
bin_names=['소득'+str(i) for i in range(7) ]
#bin_dividers는 train기준!!
train['income_total']=pd.cut(x=train['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)
test['income_total']=pd.cut(x=test['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)


In [24]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,0,0,0,소득0,Commercial associate,3,Married,Municipal apartment,-13899,-4709,0,0,0,NaN,2.0,-6.0,1.0
1,0,0,1,1,소득0,Commercial associate,2,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0,1.0
2,1,1,1,0,소득1,Working,3,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0,2.0
3,0,0,1,0,소득0,Commercial associate,2,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0,0.0
4,0,1,1,0,소득0,State servant,3,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,소득0,State servant,2,Married,House / apartment,-12079,-1984,0,0,0,Core staff,4.0,-2.0,1.0
26453,0,0,1,1,소득0,Working,3,Separated,House / apartment,-15291,-2475,0,0,0,NaN,2.0,-47.0,2.0
26454,0,1,0,0,소득1,Working,2,Civil marriage,With parents,-10082,-2015,0,0,0,Core staff,2.0,-25.0,2.0
26455,1,0,1,0,소득0,Working,1,Single / not married,House / apartment,-10145,-107,0,0,0,Laborers,1.0,-59.0,2.0


In [25]:
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
train['income_type']=label_encoder.fit_transform(train['income_type'])
test['income_type']=label_encoder.transform(test['income_type'])
########################################################################
train['family_type']=label_encoder.fit_transform(train['family_type'])
test['family_type']=label_encoder.transform(test['family_type'])
########################################################################
train['house_type']=label_encoder.fit_transform(train['house_type'])
test['house_type']=label_encoder.transform(test['house_type'])
########################################################################
train['income_total']=label_encoder.fit_transform(train['income_total'])
test['income_total']=label_encoder.fit_transform(test['income_total'])

In [26]:
train['occyp_type']=train['occyp_type'].fillna('None')
test['occyp_type']=test['occyp_type'].fillna('None')
train['occyp_type'] = train['occyp_type'].replace(['None', 'Laborers', 'Managers', 'Sales staff',
       'High skill tech staff', 'Core staff', 'Drivers', 'Medicine staff',
       'Accountants', 'Realty agents', 'Security staff', 'Cleaning staff',
       'Private service staff', 'Cooking staff', 'Secretaries',
       'HR staff', 'IT staff', 'Low-skill Laborers',
       'Waiters/barmen staff'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])
test['occyp_type'] = test['occyp_type'].replace(['None', 'Laborers', 'Managers', 'Sales staff',
       'High skill tech staff', 'Core staff', 'Drivers', 'Medicine staff',
       'Accountants', 'Realty agents', 'Security staff', 'Cleaning staff',
       'Private service staff', 'Cooking staff', 'Secretaries',
       'HR staff', 'IT staff', 'Low-skill Laborers',
       'Waiters/barmen staff'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])

In [28]:
#minus 변경하고
#구간화 함수
def make_bin(variable, n):
    train[variable]=-train[variable]
    test[variable]=-test[variable]
    count, bin_dividers =np.histogram(train[variable], bins=n) #train의 구간화를 적용
    bin_names=[str(i) for i in range(n)]
    train[variable]=pd.cut(x=train[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    test[variable]=pd.cut(x=test[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    test[variable].fillna(str(0), inplace=True) #test에는 없는 것을 임의의 값으로 채움
    ##########################################################
    train[variable]=label_encoder.fit_transform(train[variable])
    test[variable]=label_encoder.transform(test[variable])

In [29]:
make_bin('DAYS_BIRTH', n=10)
make_bin('DAYS_EMPLOYED', n=6)
make_bin('begin_month', n=4)

In [30]:
y = train['credit']
train = train.drop('credit',axis=1)

In [31]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,0,0,0,0,0,0,3,1,2,3,1,0,0,0,0,2.0,0
1,0,0,1,1,0,0,2,0,1,2,1,0,0,1,1,3.0,0
2,1,1,1,0,1,4,3,1,1,6,1,0,1,0,2,2.0,1
3,0,0,1,0,0,0,2,1,1,4,1,0,1,0,3,2.0,2
4,0,1,1,0,0,2,3,1,1,4,1,0,0,0,2,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,0,2,2,1,1,2,1,0,0,0,5,4.0,0
26453,0,0,1,1,0,4,3,2,1,4,1,0,0,0,0,2.0,3
26454,0,1,0,0,1,4,2,0,5,1,1,0,0,0,5,2.0,1
26455,1,0,1,0,0,4,1,3,1,1,1,0,0,0,1,1.0,3


In [32]:
train.to_csv('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/train_preprocess.csv', sep=',')
test.to_csv('/content/drive/MyDrive/캐글스터디/dacon신용카드사용자예측/test_preprocess.csv', sep=',')